<a href="https://colab.research.google.com/github/BoyPlankton/public_notebooks/blob/master/NBA_2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install --upgrade -q trueskill
!pip install --upgrade -q gspread
!pip install --upgrade -q gspread-dataframe

In [0]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

sht = gc.open_by_key('1COUA9BVPCUcYaUfvN3Xpjfv-8Wx--ulYK41rvnfd-H8')
worksheet = sht.worksheet("2019")

rows = worksheet.get_all_values()

import pandas as pd
from gspread_dataframe import get_as_dataframe, set_with_dataframe
import numpy as np

df = get_as_dataframe(worksheet, parse_dates=True)

In [0]:
from trueskill import Rating, rate_1vs1, BETA, quality_1vs1, global_env

import itertools
import math

# from the example at https://trueskill.org
def win_probability(team1, team2):
  delta_mu = sum(r.mu for r in team1) - sum(r.mu for r in team2)
  sum_sigma = sum(r.sigma ** 2 for r in itertools.chain(team1, team2))
  size = len(team1) + len(team2)
  denom = math.sqrt(size * (BETA * BETA) + sum_sigma)
  ts = global_env()
  return ts.cdf(delta_mu / denom)

teams = {}

def playGame(team1, team2, score1, score2):
  # it's not pretty but apply() runs the first row twice and I haven't 
  # rewritten this so that there aren't side effects
  if "counter" not in teams:
    teams["counter"] = 0
    
    return [0, 0, 0, 0, 0, 0, 0, 0]
  
  # if the teams don't exist then we need to add them.
  if team1 not in teams:
    teams[team1] = Rating()
  if team2 not in teams:
    teams[team2] = Rating()
    
  o1 = teams[team1]
  o2 = teams[team2]
  
  wp = win_probability([o1], [o2])
  
  if score1 > score2:
    n1, n2 = rate_1vs1(o1, o2)
  elif score2 > score1:
    n2, n1 = rate_1vs1(o2, o1)
  else:
    n1, n2 = rate_1vs1(o1, o2, drawn=True)
  
  teams[team1] = n1
  teams[team2] = n2
  teams["counter"] = teams["counter"] + 1
  
  return [wp, n1.mu, n1.sigma, n2.mu, n2.sigma]
  
def pandasGame(df):
  (df["win probability"],df["n_mu1"],df["n_sigma1"],df["n_mu2"],df["n_sigma2"]) = playGame(df["team1"],df["team2"],df["score1"],df["score2"])   
  
  return df


In [0]:
games_played = df[df["score1"].notna()]

In [0]:
games_played = games_played.apply(lambda x: 
                    pandasGame(x), 
                    axis=1)

In [6]:
games_played

,date,season,neutral,playoff,team1,team2,score1,score2,win probability,n_mu1,n_sigma1,n_mu2,n_sigma2
0,2018-10-16,2019,0,NaN,BOS,PHI,105.0,87.0,0.500000,29.395832,7.171476,20.604168,7.171476
1,2018-10-16,2019,0,NaN,GSW,OKC,108.0,100.0,0.500000,29.395832,7.171476,20.604168,7.171476
2,2018-10-17,2019,0,NaN,CHO,MIL,112.0,113.0,0.500000,20.604168,7.171476,29.395832,7.171476
3,2018-10-17,2019,0,NaN,ORL,MIA,104.0,101.0,0.500000,29.395832,7.171476,20.604168,7.171476
4,2018-10-17,2019,0,NaN,IND,MEM,111.0,83.0,0.500000,29.395832,7.171476,20.604168,7.171476
5,2018-10-17,2019,0,NaN,DET,BRK,103.0,100.0,0.500000,29.395832,7.171476,20.604168,7.171476
6,2018-10-17,2019,0,NaN,TOR,CLE,116.0,104.0,0.500000,29.395832,7.171476,20.604168,7.171476
7,2018-10-17,2019,0,NaN,NYK,ATL,126.0,107.0,0.500000,29.395832,7.171476,20.604168,7.171476
8,2018-10-17,2019,0,NaN,HOU,NOP,112.0,131.0,0.500000,20.604168,7.171476,29.395832,7.171476
9,2018-10-17,2019,0,NaN,SAS,MIN,112.0,108.0,0.500000,29.395832,7.171476,20.604168,7.171476


In [7]:
win_probability([teams["DAL"]], [teams["IND"]])

0.3480890233009828

In [8]:
teams

{'ATL': trueskill.Rating(mu=22.121, sigma=1.152),
 'BOS': trueskill.Rating(mu=26.527, sigma=1.098),
 'BRK': trueskill.Rating(mu=25.066, sigma=1.091),
 'CHI': trueskill.Rating(mu=20.795, sigma=1.190),
 'CHO': trueskill.Rating(mu=23.684, sigma=1.101),
 'CLE': trueskill.Rating(mu=19.879, sigma=1.222),
 'DAL': trueskill.Rating(mu=24.485, sigma=1.100),
 'DEN': trueskill.Rating(mu=28.812, sigma=1.132),
 'DET': trueskill.Rating(mu=24.522, sigma=1.104),
 'GSW': trueskill.Rating(mu=29.266, sigma=1.155),
 'HOU': trueskill.Rating(mu=27.184, sigma=1.079),
 'IND': trueskill.Rating(mu=26.865, sigma=1.115),
 'LAC': trueskill.Rating(mu=25.860, sigma=1.084),
 'LAL': trueskill.Rating(mu=24.860, sigma=1.085),
 'MEM': trueskill.Rating(mu=23.060, sigma=1.062),
 'MIA': trueskill.Rating(mu=23.795, sigma=1.097),
 'MIL': trueskill.Rating(mu=30.028, sigma=1.176),
 'MIN': trueskill.Rating(mu=24.871, sigma=1.090),
 'NOP': trueskill.Rating(mu=23.860, sigma=1.080),
 'NYK': trueskill.Rating(mu=19.506, sigma=1.199),
